In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
df = pds.random_data(size=100_000, n_cols=0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
).with_columns(
    y = pl.col("x1") * 0.5 + pl.col("x2") * 0.1 - pl.col("x3") * 0.15 # + pds.random() * 0.0001
)

In [ ]:
%%timeit
df.select(
    pds.query_lstsq(
        "x1", "x2", "x3",
        target = "y",
        # return_pred = True
        method = "l2",
        lambda_ = 0.1
    ).alias("pred")
)

In [ ]:
from sklearn import linear_model
from scipy.linalg import lstsq
x = df.select("x1", "x2", "x3").to_numpy()
y = df["y"].to_numpy()

In [ ]:
%%timeit
lstsq(x, y)

In [ ]:
%%timeit
reg = linear_model.LinearRegression(fit_intercept=False)
reg.fit(x, y)

In [ ]:
reg = linear_model.Ridge(alpha=.1)
reg.fit(x, y)

In [ ]:
%%timeit
reg = linear_model.Ridge(alpha=.1)
reg.fit(x, y)

In [ ]:
reg.coef_